In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline



In [2]:
# Replace the path with the correct path for your data.
y2015 = pd.read_csv(
    'LoanStats3d.csv',
    skipinitialspace=True,
    header=1, nrows = 50000
)

# Note the warning about dtypes.

c:\users\edgor\appdata\local\programs\python\python37-32\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
y2015.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
0,68009401,72868139,16000,16000,16000,60 months,14.85%,379.39,C,C5,...,0,2,78.9,0.0,0,2,298100,31329,281300,13400
1,68354783,73244544,9600,9600,9600,36 months,7.49%,298.58,A,A4,...,0,2,100.0,66.7,0,0,88635,55387,12500,75635
2,68466916,73356753,25000,25000,25000,36 months,7.49%,777.55,A,A4,...,0,0,100.0,20.0,0,0,373572,68056,38400,82117
3,68466961,73356799,28000,28000,28000,36 months,6.49%,858.05,A,A2,...,0,0,91.7,22.2,0,0,304003,74920,41500,42503
4,68495092,73384866,8650,8650,8650,36 months,19.89%,320.99,E,E3,...,0,12,100.0,50.0,1,0,38998,18926,2750,18248


In [4]:
categorical = y2015.select_dtypes(include=['object'])
for i in categorical:
    column = categorical[i]
    print(i)
    print(column.nunique())

term
2
int_rate
70
grade
7
sub_grade
35
emp_title
21285
emp_length
11
home_ownership
4
verification_status
3
issue_d
2
loan_status
7
pymnt_plan
1
url
50000
desc
3
purpose
12
title
12
zip_code
856
addr_state
49
earliest_cr_line
605
revol_util
1083
initial_list_status
2
last_pymnt_d
14
next_pymnt_d
3
last_credit_pull_d
15
application_type
2
verification_status_joint
3


In [5]:
y2015 = y2015[:-2]

In [6]:
# Convert ID and Interest Rate to numeric.
y2015['id'] = pd.to_numeric(y2015['id'], errors='coerce')
y2015['int_rate'] = pd.to_numeric(y2015['int_rate'].str.strip('%'), errors='coerce')

# Drop other columns with many unique variables
y2015.drop(['url', 'emp_title', 'zip_code', 'earliest_cr_line', 'revol_util',
            'sub_grade', 'addr_state', 'desc'], 1, inplace=True)

In [7]:
y2015 = y2015[:-2]

In [8]:
from sklearn import ensemble
from sklearn.model_selection import cross_val_score

rfc = ensemble.RandomForestClassifier()
X = y2015.drop('loan_status', 1)
Y = y2015['loan_status']
X = pd.get_dummies(X)
X = X.dropna(axis=1)

cross_val_score(rfc, X, Y, cv=10)

c:\users\edgor\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


array([0.98280688, 0.98180728, 0.980008  , 0.9820036 , 0.98180364,
       0.98039608, 0.98039216, 0.979992  , 0.98339336, 0.98038038])

In [9]:

X_prime = X[['out_prncp_inv','last_pymnt_amnt','out_prncp','total_rec_prncp','last_pymnt_d_Jan-2017','last_credit_pull_d_Oct-2016','total_pymnt','last_credit_pull_d_Jan-2017','total_pymnt_inv','last_pymnt_d_Oct-2016']]

In [11]:
features = list(X_prime.columns)

for i in range(len(features),0,-1): 
    X = X[features[:i+1]]
    print('Run Number: {}.\n'.format(len(features)-i+1))
    print('Number of Features: {}.\n'.format(i))
    print('Dropped Feature(s): \n{}\n'.format(features[i:]))
    print("Average Accuracy (10-Fold Cross-Validation): {:.2%}.\n".format(cross_val_score(rfc, X_prime, Y, cv=10).mean()))

Run Number: 1.

Number of Features: 10.

Dropped Feature(s): 
[]

Average Accuracy (10-Fold Cross-Validation): 97.43%.

Run Number: 2.

Number of Features: 9.

Dropped Feature(s): 
['last_pymnt_d_Oct-2016']

Average Accuracy (10-Fold Cross-Validation): 97.38%.

Run Number: 3.

Number of Features: 8.

Dropped Feature(s): 
['total_pymnt_inv', 'last_pymnt_d_Oct-2016']

Average Accuracy (10-Fold Cross-Validation): 97.38%.

Run Number: 4.

Number of Features: 7.

Dropped Feature(s): 
['last_credit_pull_d_Jan-2017', 'total_pymnt_inv', 'last_pymnt_d_Oct-2016']

Average Accuracy (10-Fold Cross-Validation): 97.37%.

Run Number: 5.

Number of Features: 6.

Dropped Feature(s): 
['total_pymnt', 'last_credit_pull_d_Jan-2017', 'total_pymnt_inv', 'last_pymnt_d_Oct-2016']

Average Accuracy (10-Fold Cross-Validation): 97.40%.

Run Number: 6.

Number of Features: 5.

Dropped Feature(s): 
['last_credit_pull_d_Oct-2016', 'total_pymnt', 'last_credit_pull_d_Jan-2017', 'total_pymnt_inv', 'last_pymnt_d_Oct-20

I was only able to reduce the accuracy to 97%. All the features were related to payment amount or outstanding principal. 